<a href="https://colab.research.google.com/github/Ino54/MA_GreenAI-Practical-Experiments/blob/main/deepseek_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---------- Requirements ----------
%%writefile requirements.txt
transformers>=4.44
accelerate>=0.33
bitsandbytes
datasets>=2.20
evaluate>=0.4
sacrebleu>=2.4
codecarbon>=2.5,<3
pynvml>=12,<13
psutil
numpy==2.0.2
pandas==2.2.2
huggingface_hub

Writing requirements.txt


In [ ]:
!pip -q install -U -r requirements.txt --no-warn-conflicts
!pip uninstall -y -q google-genai firebase-admin || true

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 128.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.6/517.6 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.2/291.2 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ---------- Hugging Face Login via Colab-Secret ----------
from google.colab import userdata
from huggingface_hub import login
hf_token = userdata.get("HF_TOKEN")
if hf_token:
    login(hf_token); print("Hugging Face Login erfolgreich!")
else:
    print("WARNUNG: Kein HF_TOKEN gefunden")

Hugging Face Login erfolgreich!


In [ ]:
# ---------- Login, Drive, Ordner ----------
import os, shutil, time, pathlib, platform, gc, re, math, warnings, inspect
warnings.filterwarnings("ignore")
from google.colab import drive
MOUNTPOINT="/content/drive"
already=os.path.isdir(os.path.join(MOUNTPOINT,"MyDrive"))
if not already and os.path.isdir(MOUNTPOINT) and os.listdir(MOUNTPOINT):
    backup=f"/content/drive_stale_{int(time.time())}"
    shutil.move(MOUNTPOINT, backup); os.makedirs(MOUNTPOINT, exist_ok=True)
drive.mount(MOUNTPOINT, force_remount=(not already))

work_dir="/content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/deepseek_frameworks"
pathlib.Path(work_dir).mkdir(parents=True, exist_ok=True)
os.chdir(work_dir); project_dir=work_dir
print("Arbeitsordner:", os.getcwd())

Mounted at /content/drive
Arbeitsordner: /content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/deepseek_frameworks


In [ ]:
# ---------- Imports & Device ----------
import numpy as np, pandas as pd, psutil, torch
from contextlib import nullcontext
from types import SimpleNamespace
from datasets import load_dataset
import evaluate

from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          BitsAndBytesConfig, GenerationConfig, set_seed)
from codecarbon import EmissionsTracker

import os as _os
_os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"

set_seed(42)
device="cuda" if torch.cuda.is_available() else "cpu"
if device=="cuda":
    gpu_name=torch.cuda.get_device_name(0)
    vram_total_gb=torch.cuda.get_device_properties(0).total_memory/(1024**3)
    torch.backends.cuda.matmul.allow_tf32=True
    torch.backends.cudnn.benchmark=True
    try: torch.set_float32_matmul_precision("high")
    except: pass
else:
    gpu_name="CPU"; vram_total_gb=0.0
print(f"Device: {device} | GPU: {gpu_name} | VRAM={vram_total_gb:.1f} GB | Torch {torch.__version__} | Py {platform.python_version()}")

RESULT_BASENAME="deepseek_framew_4bit"
ALIAS="r1q15b"
MODEL_ID="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
ATTN_IMPL="sdpa"  # Qwen/DeepSeek: SDPA performant
USE_BF16=(device=="cuda" and torch.cuda.get_device_capability(0)[0] >= 8)

Device: cuda | GPU: NVIDIA A100-SXM4-40GB | VRAM=39.6 GB | Torch 2.8.0+cu126 | Py 3.12.11


In [ ]:
# ---------- Eval-Konfiguration ----------
EVAL = {
    "max_new_tokens": 32,
    "ppl":  {"name":"wikitext","config":"wikitext-2-raw-v1","split":"test[:1%]"},
    "bleu": {"name":"wmt14","config":"de-en","split":"test[:32]"},
}
PROMPTS = [
    "Schreibe einen kurzen Absatz über nachhaltige KI.",
    "Erkläre in einfachen Worten, was Quantisierung in neuronalen Netzen ist.",
    "Nenne drei Vorteile von Mixture-of-Experts-Modellen."
]
bleu_metric = evaluate.load("sacrebleu")

MAX_LEN_CAP = 128 # verringern auf 192/128 falls VRAM knapp
GENCFG = GenerationConfig(do_sample=False, temperature=None, top_p=None, top_k=None, num_beams=1)

In [ ]:
# ---------- CodeCarbon-Helfer ----------
def _cleanup_cc_locks():
    for p in ["/tmp/.codecarbon.lock",
              _os.path.expanduser("~/.codecarbon/codecarbon.lock"),
              "/content/.codecarbon/codecarbon.lock"]:
        try:
            if _os.path.exists(p): _os.remove(p)
        except: pass

_os.environ["CODECARBON_CACHE_DIR"]=f"/content/.cc_cache_framew_{int(time.time())}"

def _cc_supported_kwargs():
    base=dict(log_level="error", output_dir=".", measure_power_secs=1, tracking_mode="process")
    try:
        params=inspect.signature(EmissionsTracker.__init__).parameters
        if "allow_multiple_runs" in params: base["allow_multiple_runs"]=True
        if "cloud_provider" in params:      base["cloud_provider"]="google"
        if "cloud_region" in params:        base["cloud_region"]="europe-west10"
        if "country_iso_code" in params:    base["country_iso_code"]="DEU"
    except: pass
    return base
def make_trk(name,out):
    _cleanup_cc_locks()
    return EmissionsTracker(project_name=name, output_file=out, **_cc_supported_kwargs())
def start(tr):
    try: tr.start(); return True
    except:
        _cleanup_cc_locks()
        try: tr.start(); return True
        except: return False
def stop(tr, st):
    if not st: return SimpleNamespace(energy_consumed=0.0, emissions=0.0)
    try: return tr.stop()
    except: return SimpleNamespace(energy_consumed=0.0, emissions=0.0)
def unpack(em):
    if hasattr(em,"energy_consumed") and hasattr(em,"emissions"):
        try: return float(em.energy_consumed), float(em.emissions)
        except: return 0.0,0.0
    if isinstance(em, dict):
        e=em.get("energy_consumed",0.0); c=em.get("emissions", em.get("emissions_kg",0.0))
        try: return float(e), float(c)
        except: return 0.0,0.0
    try: return 0.0, float(em)
    except: return 0.0,0.0
def read_energy(path):
    try:
        if not _os.path.exists(path): return 0.0
        import pandas as _pd
        df=_pd.read_csv(path)
        for c in ["energy_consumed","energy_consumed_kwh","energy_consumed (kWh)","energy (kWh)"]:
            if c in df.columns: return float(df[c].iloc[-1])
        for c in df.columns:
            if "energy" in c.lower() and "kwh" in c.lower(): return float(df[c].iloc[-1])
    except: pass
    return 0.0
def measure(phase, fn, prefix):
    logfile=_os.path.join(project_dir, f"{prefix}_{phase}.csv")
    tr=make_trk(f"{prefix}_{phase}", logfile)
    import time as _t
    st=start(tr); t0=_t.time(); res=fn(); t1=_t.time()
    em=stop(tr, st); ekwh,co2=unpack(em)
    if ekwh==0.0:
        ek=read_energy(logfile)
        if ek: ekwh=ek
    return {"phase":phase,"time_s":t1-t0,"energy_kwh":ekwh,"co2_kg":co2}, res

In [ ]:
# ---------- Utilities ----------
def safe_max_len(tok, model, fallback=2048, upper=100000):
    cand=getattr(tok,"model_max_length",None)
    if isinstance(cand,int) and 0<cand<upper: return min(cand, fallback)
    cand=getattr(getattr(model,"config",None),"max_position_embeddings",None)
    if isinstance(cand,int) and 0<cand<upper: return min(cand, fallback)
    return fallback

def autocast_ctx():
    if device!="cuda": return nullcontext()
    return torch.autocast(device_type="cuda", dtype=(torch.bfloat16 if USE_BF16 else torch.float16))

def _pin_and_move(batch, device):
    out={}
    for k,v in batch.items():
        if isinstance(v, torch.Tensor):
            try:
                if v.device.type=="cpu": v=v.pin_memory()
            except: pass
            out[k]=v.to(device, non_blocking=True)
        else:
            out[k]=v
    return out

def get_model_device(model):
    try: return next(model.parameters()).device
    except StopIteration: return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def capture_memory():
    ram=psutil.Process().memory_info().rss
    valloc=torch.cuda.memory_allocated() if torch.cuda.is_available() else 0
    vres =torch.cuda.memory_reserved()  if torch.cuda.is_available() else 0
    return ram, valloc, vres

def bytes_to_gb(b): return float(b)/(1024**3)

In [ ]:
# ---------- HF-Backend: 4-bit (NF4) ----------
def load_hf_4bit(model_id:str):
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    if tok.pad_token_id is None: tok.pad_token = tok.eos_token
    tok.padding_side="left"

    if torch.cuda.is_available(): torch.cuda.empty_cache(); gc.collect()

    bnb4 = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=(torch.bfloat16 if USE_BF16 else torch.float16),
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_id, device_map="auto", quantization_config=bnb4, attn_implementation=ATTN_IMPL
    ).eval()
    return tok, model, "hf_4bit_sdpa"

In [ ]:
# ---------- Eval: Generation / PPL / BLEU ----------
def simple_generate(model, tok, prompts, max_new_tokens=32):
    dev=get_model_device(model)
    ml=min(MAX_LEN_CAP, safe_max_len(tok, model))
    enc=tok(prompts, return_tensors="pt", truncation=True, max_length=ml, padding=True)
    if dev.type=="cuda": enc=_pin_and_move(enc, dev)
    room=ml-enc["input_ids"].shape[1]; cur_new=max(1, min(max_new_tokens, int(room)))
    with torch.inference_mode(), autocast_ctx():
        out=model.generate(**enc, max_new_tokens=cur_new, generation_config=GENCFG, pad_token_id=tok.eos_token_id)

    texts, total_gen_tokens=[], 0
    max_in=int(enc["input_ids"].shape[1]); pad_id=tok.pad_token_id; eos_id=tok.eos_token_id
    for i in range(out.size(0)):
        seq=out[i]; gen_slice=seq[max_in:]; gen_i=0
        for t in gen_slice.tolist():
            if t==eos_id: gen_i+=1; break
            if (pad_id is not None) and (t==pad_id): break
            gen_i+=1
        total_gen_tokens+=gen_i
        texts.append(tok.decode(seq, skip_special_tokens=True))
    return texts, total_gen_tokens

import torch.nn.functional as F
def eval_perplexity(model, tok, ds_cfg):
    dev=get_model_device(model)
    ds=load_dataset(ds_cfg["name"], ds_cfg["config"], split=ds_cfg["split"])
    ml=min(MAX_LEN_CAP, safe_max_len(tok, model))
    losses=[]
    with torch.inference_mode(), autocast_ctx():
        for t in ds["text"]:
            if not isinstance(t,str) or len(t.strip())<4: continue
            enc=tok(t, return_tensors="pt", truncation=True, max_length=ml)
            if dev.type=="cuda": enc=_pin_and_move(enc, dev)
            out=model(enc["input_ids"], labels=enc["input_ids"]); loss=out.loss
            losses.append(float(loss.detach().cpu()))
    return math.exp(np.mean(losses)) if losses else None

def eval_bleu_llm(model, tok, ds_cfg, max_new_tokens=32, batch_size=8):
    dev=get_model_device(model)
    ds=load_dataset(ds_cfg["name"], ds_cfg["config"], split=ds_cfg["split"])
    ml=min(MAX_LEN_CAP, safe_max_len(tok, model))
    preds, refs=[], []
    with torch.inference_mode(), autocast_ctx():
        batch_prompts, batch_refs=[], []
        for ex in ds:
            de,en=ex["translation"]["de"], ex["translation"]["en"]
            prompt=f"Translate to English:\nGerman: {de}\nEnglish:"
            batch_prompts.append(prompt); batch_refs.append(en)
            if len(batch_prompts)>=batch_size:
                enc=tok(batch_prompts, return_tensors="pt", truncation=True, max_length=ml, padding=True)
                if dev.type=="cuda": enc=_pin_and_move(enc, dev)
                room=ml-enc["input_ids"].shape[1]; cur_new=max(1, min(max_new_tokens, int(room)))
                out=model.generate(**enc, max_new_tokens=cur_new, generation_config=GENCFG, pad_token_id=tok.eos_token_id)
                for i in range(out.size(0)):
                    gen=tok.decode(out[i], skip_special_tokens=True)
                    hyp=gen.split("English:")[-1].strip().split("\n")[0].strip() or gen.strip()
                    preds.append(hyp); refs.append([batch_refs[i]])
                batch_prompts, batch_refs=[], []
        if batch_prompts:
            enc=tok(batch_prompts, return_tensors="pt", truncation=True, max_length=ml, padding=True)
            if dev.type=="cuda": enc=_pin_and_move(enc, dev)
            room=ml-enc["input_ids"].shape[1]; cur_new=max(1, min(max_new_tokens, int(room)))
            out=model.generate(**enc, max_new_tokens=cur_new, generation_config=GENCFG, pad_token_id=tok.eos_token_id)
            for i in range(out.size(0)):
                gen=tok.decode(out[i], skip_special_tokens=True)
                hyp=gen.split("English:")[-1].strip().split("\n")[0].strip() or gen.strip()
                preds.append(hyp); refs.append([batch_prompts and batch_refs[i]])
    return float(bleu_metric.compute(predictions=preds, references=refs)["score"])

In [ ]:
# ---------- Runner (HF 4-bit) ----------
def run_backend(model_id:str, alias:str):
    print(f"\n### Starte Framework (HF 4-bit): {alias} ({model_id})")

    def _do_load():
        tok, model, note = load_hf_4bit(model_id)
        dev=get_model_device(model); ml=min(MAX_LEN_CAP, safe_max_len(tok, model))
        wenc=tok(["Warmup token 1","Warmup token 2"], return_tensors="pt",
                 truncation=True, max_length=ml, padding=True)
        if dev.type=="cuda": wenc=_pin_and_move(wenc, dev)
        with torch.inference_mode(), autocast_ctx():
            _=model.generate(**wenc, max_new_tokens=1, generation_config=GENCFG, pad_token_id=tok.eos_token_id)
        return tok, model, note

    m_warm, (tok, model, backend_note)=measure("warmup", _do_load, f"{RESULT_BASENAME}_{alias}")

    m_gen, (samples, n_tok)=measure("gen",
        lambda: simple_generate(model, tok, PROMPTS, EVAL["max_new_tokens"]),
        f"{RESULT_BASENAME}_{alias}"
    )
    if torch.cuda.is_available(): torch.cuda.empty_cache(); gc.collect()

    m_ppl, ppl=measure("ppl",
        lambda: eval_perplexity(model, tok, EVAL["ppl"]),
        f"{RESULT_BASENAME}_{alias}"
    )
    if torch.cuda.is_available(): torch.cuda.empty_cache(); gc.collect()

    m_bleu, bleu=measure("bleu",
        lambda: eval_bleu_llm(model, tok, EVAL["bleu"], EVAL["max_new_tokens"], batch_size=8),
        f"{RESULT_BASENAME}_{alias}"
    )
    if torch.cuda.is_available(): torch.cuda.empty_cache(); gc.collect()

    total_time=m_warm["time_s"]+m_gen["time_s"]+m_ppl["time_s"]+m_bleu["time_s"]
    total_energy=m_warm["energy_kwh"]+m_gen["energy_kwh"]+m_ppl["energy_kwh"]+m_bleu["energy_kwh"]
    total_co2=m_warm["co2_kg"]+m_gen["co2_kg"]+m_ppl["co2_kg"]+m_bleu["co2_kg"]

    ram,valloc,vres=capture_memory()
    per_phase=[m_warm,m_gen,m_ppl,m_bleu]
    for p in per_phase:
        p["alias"]=alias; p["model_id"]=model_id; p["backend"]=backend_note

    row=dict(
        model_id=model_id, alias=alias, backend=backend_note,
        precision="int4_nf4_sdpa",
        time_s=total_time, energy_kwh=total_energy, co2_kg=total_co2,
        kg_per_kwh=(total_co2/total_energy) if total_energy else None,
        tokens_out=int(n_tok), ppl=ppl, bleu=bleu,
        ram_GB=bytes_to_gb(ram), vram_alloc_GB=bytes_to_gb(valloc), vram_reserved_GB=bytes_to_gb(vres),
        notes=f"MAX_LEN_CAP={MAX_LEN_CAP}; bf16_autocast={USE_BF16}"
    )

    out_samples=os.path.join(project_dir, f"{RESULT_BASENAME}_samples_{alias}.txt")
    with open(out_samples,"w",encoding="utf-8") as f:
        for i,txt in enumerate(samples,1):
            f.write(f"--- Beispiel {i} ({alias}, {backend_note}) ---\n{txt}\n\n")
    print("Beispiele gespeichert:", out_samples)

    return row, per_phase

In [ ]:
# ---------- Ausführen & Speichern ----------
row, phases = run_backend(MODEL_ID, ALIAS)

df  = pd.DataFrame([row]).sort_values(["backend"]).reset_index(drop=True)
dfp = pd.DataFrame(phases)
dfp["wh_total"]=dfp["energy_kwh"]*1000.0
dfp["phase"]=pd.Categorical(dfp["phase"], categories=["warmup","gen","ppl","bleu"], ordered=True)

out_csv = os.path.join(project_dir, f"{RESULT_BASENAME}_results.csv")
df.to_csv(out_csv, index=False)
out_phase_csv = os.path.join(project_dir, f"{RESULT_BASENAME}_per_phase.csv")
dfp.to_csv(out_phase_csv, index=False)

print("\nErgebnisse (gesamt):")
print(df); print("Gespeichert (gesamt):", out_csv)

print("\nPer-Phase Übersicht:")
print(dfp[["backend","phase","time_s","energy_kwh","co2_kg"]].sort_values(["backend","phase"]))
print("Gespeichert (per Phase):", out_phase_csv)

[codecarbon WARNING @ 11:21:27] Multiple instances of codecarbon are allowed to run at the same time.



### Starte Framework (HF 4-bit): r1q15b (deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

...

de-en/test-00000-of-00001.parquet:   0%|          | 0.00/509k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

Beispiele gespeichert: /content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/deepseek_frameworks/deepseek_framew_4bit_samples_r1q15b.txt

Ergebnisse (gesamt):
                                    model_id   alias       backend  \
0  deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B  r1q15b  hf_4bit_sdpa   

       precision     time_s  energy_kwh    co2_kg  kg_per_kwh  tokens_out  \
0  int4_nf4_sdpa  67.500892    0.001906  0.000863    0.452621          96   

          ppl      bleu    ram_GB  vram_alloc_GB  vram_reserved_GB  \
0  238.563029  11.30959  4.244473       1.517489          1.583984   

                                 notes  
0  MAX_LEN_CAP=128; bf16_autocast=True  
Gespeichert (gesamt): /content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/deepseek_frameworks/deepseek_framew_4bit_results.csv

Per-Phase Übersicht:
        backend   phase     time_s  energy_kwh    co2_kg
0  hf_4bit_sdpa  warmup  30.314054    0.000776  0.000351
1  hf_4bit_sdpa     gen   3.138607    0.000